# Agents SDK experimental de OpenAI

In [ ]:
%cd /content/drive/MyDrive/Idas/AI_Agents_Bootcamp_Zero_to_Mastery/OpenAI_Agents_SDK/AI_Sommelier

/content/drive/MyDrive/AI Agents/OpenAI Agents SDK/Digital Waiter


In [ ]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get("genai_course")

In [ ]:
import asyncio # while the agent thinks or searchs for vecto, the code does not stop
from uuid import uuid4

from openai import OpenAI
from agents import ( # we are not just going to use chat.completions
    Agent,
    Runner, # starts the loop
    trace,
    ItemHelpers,
    MessageOutputItem,
    HandoffOutputItem,
    FileSearchTool, # pre-build for RAG to search documents
    function_tool,
)
from agents.extensions.handoff_prompt import RECOMMENDED_PROMPT_PREFIX # security instruction to behave the agent

client = OpenAI()

# Upload the file to OpenAI

https://platform.openai.com/docs/guides/agents-sdk

In [ ]:
# Upload a file once, reuse everywhere
# https://platform.openai.com/docs/api-reference/files/create
path = 'Vinhos_baba_d_urso.pdf'
upload = client.files.create(# Upload the file to the OpenAI servers
    file = open(path, 'rb'),
    purpose = 'assistants'
)

In [ ]:
# Retrieve and print the file ID
FILE_ID = upload.id
FILE_ID

Vector Stores

In [ ]:
# Create the vector store with code
vector_store = client.vector_stores.create(
    name = 'Vector_Store_X'
)

In [ ]:
# Insert the file in the vector store
# Create the vector store at openAI
vs_file = client.vector_stores.files.create(
    vector_store_id = vector_store.id, # create the vector ID from OpenAI
    file_id = FILE_ID
)
# with this we just have chunk and embedded the PDG

# AI Sommelier Agent


In [ ]:
# define the instructions and other configs
instructions = f"""{RECOMMENDED_PROMPT_PREFIX}
You are Bitte's AI Sommelier:
Your task is to provide the user with wine recomendation for their dish.
Use FileSearchTool to pick up to 3 wines from the vector store.
If the interaction has nothing to do with the wine, do the handoff.
"""
# do the handoff significa escalar al jefe

# Put your vector ID here
vs_id = vector_store.id

In [ ]:
# Escalate to Boss
# Define to escalate to boss function
@function_tool(
    name_override = 'escalate_to_boss',
    description_override = 'Talk to the guest about wine. If the topic is not wine, escalate to the boss'
)

async def escalate_to_boss(request_str) -> str:
  print(f'Escalating to the boss {request}')
  return "The boss will be here soon"

In [ ]:
# Build the AI Agent
sommelier = Agent(
    name = 'Bitte AI Sommelier',
    model = 'gpt-4o-mini',
    instructions = instructions,
    tools = [
        FileSearchTool(vector_store_ids = [vs_id],
                       max_num_results = 3),
        escalate_to_boss # esta herramienta es un tool ahora
    ]
)

Running the AI Agent

In [ ]:
# Build he Chat interactions
async def chat():
  # Store the conversation
  history = []

  # while loop for the conversation (not to have a conversation for ever)
  # bucle infinito hasta que el usuario salga
  while True:
    user = input('User: ')
    if user.lower() in {'exit', 'quit'}:
      break

    history.append({'role': 'user', 'content': user})

    # Apply tracing to the conversation
    with trace('Bitte Sommelier', group_id = uuid4().hex):
      run = await Runner.run(sommelier, history)

    stop_chat = False

    for itm in run.new_items:
      if isinstance(itm, MessageOutputItem): # If the agent decides to talk
        print('Sommelier', ItemHelpers.text_message_output(itm))

      elif isinstance(itm, ToolCallOutputItem): # if the agent use a tool
        # The Agent decides to use the tool escalate_to_boss
        print('Boss was summoned')
        stop_chat = True
        break

    if stop_chat:
      break

    history = run.to_input_list()

# Run the chat
 await chat()